In [1]:
import tensorflow as tf
import os
import sys
import go_datafile_reader
import model
import numpy as np
import model_eval
reload(model)
reload(go_datafile_reader)
reload(model_eval)

<module 'model_eval' from 'model_eval.pyc'>

In [2]:
train_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/train"
test_dir = "/home/justin/Programming/GoAI/MovePredictionCNN/data/input/test"
ckpt_path = "/home/justin/Programming/GoAI/MovePredictionCNN/data/working/board_eval_cnn_5layer.ckpt"
RECORD_BYTES = 5

In [3]:
train_files = []
for subdir, dirs, files in os.walk(train_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            train_files.append(filepath)

test_files = []
for subdir, dirs, files in os.walk(test_dir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".dat"):
            test_files.append(filepath)

print "num train: %d, num test: %d" %(len(train_files), len(test_files))

num train: 14724, num test: 149


In [4]:
reader = go_datafile_reader.GoDatafileReader(train_files)
batch_aggregator = go_datafile_reader.BatchAggregator(reader, mega_batch_size=10000)
test_reader = go_datafile_reader.GoDatafileReader(test_files)

test_reader.num_epochs = 0
test_features = []
test_targets = []
while(test_reader.num_epochs == 0):
    final_state, _, feature_cube = test_reader.read_sample()
    test_features.append(feature_cube)
    test_targets.append(final_state)

In [5]:
x, ownership = model.place_holders()
y_conv = model.model(x)
loss = model.loss_function(ownership, y_conv)
train_op = model.train_step(loss)

prediction = tf.round(y_conv)
correct_prediction = tf.equal(ownership, prediction)
correct_count = tf.reduce_sum(tf.cast(correct_prediction, "float"))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver(tf.all_variables())
#saver.restore(sess, ckpt_path)

In [14]:
10000/50

200

In [7]:
training_accuracies = []
for k in range(100000):
    x_batch, y_batch = batch_aggregator.get_batch(50)
    _, loss_value, y_value = sess.run([train_op, loss, y_conv], feed_dict={x:x_batch, ownership:y_batch})
    if k % 10 == 0:
        acc = accuracy.eval(feed_dict={x:x_batch, ownership:y_batch})
        training_accuracies.append(acc)
        print loss_value, acc, batch_aggregator.go_file_reader.index_of_file, \
            batch_aggregator.go_file_reader.num_epochs
    if k % 1000 == 0:
        test_accuracy = model_eval.test_accuracy(test_features, test_targets, x, ownership, correct_count)
        print "Test accuracy: %f" %test_accuracy
    if k % 1000 == 0:
        print "Saving Checkpoint..."
        saver.save(sess, ckpt_path)
    

 8822.79 0.508033 37 0
Test accuracy: 0.507519
Saving Checkpoint...
7639.79 0.568089 37 0
6724.45 0.61374 37 0
5493.75 0.676344 37 0
5487.39 0.676842 37 0
4844.65 0.712687 37 0
4662.69 0.71856 37 0
4784.21 0.713463 37 0
4635.97 0.71867 37 0
4359.38 0.73795 37 0
4303.27 0.741053 37 0
4035.07 0.752355 37 0
4185.78 0.741385 37 0
3934.91 0.76072 37 0
4264.85 0.739945 37 0
4149.23 0.747701 37 0
3814.3 0.762382 37 0
4006.18 0.751468 37 0
3892.1 0.761551 37 0
3583.85 0.779945 37 0
Got to end of mega batch, reinitializing...
Epoch = 0
4265.03 0.739778 74 0
4029.26 0.752465 74 0
3821.15 0.769862 74 0
4141.71 0.745596 74 0
4138.81 0.746925 74 0
3937.7 0.755734 74 0
3981.65 0.752742 74 0
3991.68 0.753684 74 0
3761.11 0.765596 74 0
3606.41 0.774737 74 0
4033.95 0.747701 74 0
3592.05 0.780277 74 0
3391.05 0.788033 74 0
3292.49 0.797175 74 0
3540.6 0.778449 74 0
3730.07 0.766316 74 0
3719.96 0.766704 74 0
3705.09 0.769086 74 0
3756.05 0.762936 74 0
3667.92 0.768753 74 0
Got to end of mega batch, rei

KeyboardInterrupt: 

In [19]:
import matplotlib.pyplot as plt
import numpy as np

In [22]:
def rolling_mean(number_list, window = 20):
    means = np.zeros(len(number_list))
    for i in xrange(len(means)):
        sub_window = number_list[i-window+1:i+1]
        means[i] = np.mean(sub_window)
    return means

In [26]:
means = rolling_mean(training_accuracies, 200)
plt.plot(training_accuracies, label="Batch Accuracy")
plt.plot(means, label= "Rolling Mean")
plt.title("Bad Data Mixing")
plt.xlabel("Batch Num / 10")
plt.ylabel("Accuracy")
plt.legend(loc=3)
plt.show()

In [7]:
x_batch, y_batch = batch_aggregator.get_batch(50)
y_val, y_preds = sess.run([y_conv, prediction], feed_dict={x:x_batch, ownership:y_batch})

Got to end of mega batch, reinitializing...


In [8]:
idx = 5
y_pred = np.reshape(y_preds[idx], [19,19])
y_true = np.reshape(y_batch[idx], [19,19])
feature_cube = x_batch[idx]

model_util.print_info(feature_cube, y_pred, y_true)

    

*******************   1100111011100000000   1111111100000000000
***1***************   1111000111110000000   1111111110000000000
**01001*******0****   1101001111101000000   1111111100000000000
**1001*1*1******0**   1110001111111000000   1111111111000000000
**11001************   0111001101100100000   1111111111000000000
*1**10*************   1110001111110100000   1111111111000000000
**110**************   1111010101101000000   1111011111100000000
**10************0**   1110000111110111000   1110001000100000000
***0***************   1110010010111010100   1100000000000000011
**10************1**   1110000001011011111   1110000000000000111
**100**************   1110000001001001111   1110000000000001111
**1101*********01**   1111000110000001111   0011000000000100111
****10*0*0*0**011**   1010000000000001111   0000000000001101111
*0000*10*10**0*01**   0000000001000001111   0000001000001111111
**1*1*010***1*0110*   1011111111111001111   0011111100011111111
**010**1111***0001*   100111111111100001

In [86]:
old_index = 0
old_k = 0
for k in range(40000):
    final_state, _, feature_cube = reader.read_sample()
    if reader.index_of_file != old_index:
        print k-old_k
        old_k = k
        old_index = reader.index_of_file    

0
208
267
266
252
252
269
294
240
252
277
269
285
260
274
328
309
238
314
270
253
283
245
242
324
238
266
223
235
235
254
298
318
251
258
267
241
259
231
279
312
246
296
221
286
243
266
256
241
308
241
294
292
279
276
345
265
287
252
264
258
267
238
254
243
265
224
265
301
262
225
235
231
314
316
267
322
277
246
262
242
280
230
280
290
228
240
261
314
306
238
258
272
291
249
261
249
261
271
255
288
258
242
246
301
285
324
265
337
363
254
249
220
237
237
264
246
244
244
297
302
285
305
275
261
257
244
242
280
251
290
268
333
296
266
284
263
287
321
288
271
263
257
287
249
236
342
282
265
